### Tuesday, January 2, 2024

Trying this again ... using the hfpt_Dec14 container.

Yup! It still works!

### Wednesday, December 27, 2023

Trying this again ... using the hfpt_Dec14 container. 

I am going to torch all the models in the /root/.cache/huggingface/hub directory to see how much space gets freed. 

Data2 is at 355.3 GB currently ... now let's clear away all the current models and see what we gain ... hmm odd, nothing seems to have been gained ... sigh.

Right .... have to run 

1) pip install auto-gptq 
2) pip install optimum


### Friday, November 17, 2023

Been doing some model cleanup on the hfpt_Oct28 container, and want to re-run this just to ensure all is still well ...

Yup! This still works! Nice!

### Thursday, November 16, 2023

https://huggingface.co/TheBloke/CodeLlama-34B-Instruct-GPTQ

Let's see if we can get this to run locally, shall we ... 

I am trying this model because it has the most downloads of all his models, AND of all the huggingface models that are 'code' related.

Wow! This actually works on the 4090! I am stunned! 20524MiB of VRAM!


In [1]:
!ls /root/.cache/huggingface/hub

models--TheBloke--Llama-2-13b-Chat-GPTQ
models--microsoft--Orca-2-13b
models--mistralai--Mistral-7B-Instruct-v0.2
version.txt


In [4]:
!mkdir /root/.cache/huggingface/hub

In [2]:
!rm -rf /root/.cache/huggingface/hub

In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 18.3GB to c8324b70601d://root/.cache/huggingface/hub

In [2]:
!ls /root/.cache/huggingface/hub

models--TheBloke--CodeLlama-34B-Instruct-GPTQ
models--TheBloke--Llama-2-13b-Chat-GPTQ
models--microsoft--Orca-2-13b
models--mistralai--Mistral-7B-Instruct-v0.2
version.txt


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 18.3GB to /home/rob/Data3/huggingface/transformers


# (base) rob@KAUWITB:~$ docker cp /home/rob/Data3/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ c9b676310ea0://home/rob/Data2/huggingface/transformers
# Successfully copied 18.3GB to c9b676310ea0://home/rob/Data2/huggingface/transformers
# (base) rob@KAUWITB:~$ 


In [3]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# !ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  version.txt


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [5]:
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

# 15.1s 18740 MiB VRAM ... why less than before??

# 16.7s to load, takes up 20132Mib VRAM

In [6]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

# 20.0s 18974 MiB VRAM

# 6.1s 18856 MiB VRAM

# 22.5s 20524Mib VRAM




*** Generate:
<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

The prompt is asking for a description of AI, so here is a possible answer:

AI stands for artificial intelligence, which refers to the ability of a computer or machine to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision making, and language translation. AI systems can analyze data, identify patterns, and make predictions or decisions based on that data. They can also learn and improve their performance over time, much like humans do.

There are several types of AI, including:

1. Narrow or weak AI: This type of AI is designed to perform a specific task, such as playing chess or recognizing faces.
2. General or strong AI: This type of AI is designed to perform any intellectual task that a human can, such as reasoning, prob

In [7]:
# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 18.6s 18974 MiB VRAM

# 7.2s 18878 MiB VRAM

# 11.9s 20524Mib VRAM

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

The first line of this problem is a classic trick question. The correct answer is "AI is not AI". Here's why:

"AI" stands for "Artificial Intelligence", which refers to the ability of a computer or machine to perform tasks that typically require human intelligence, such as learning, reasoning, and decision-making. However, the term "AI" does not refer to any specific algorithm or technique used to achieve these tasks. It is simply a label that encompasses a wide range of approaches and techniques.

Therefore, when asked to write code that solves the problem "Tell me about AI", it is impossible to do so without being vague or unspecific. The question is too broad and open-ended, and there are many different ways to approach it.

One possible solution would be to write a program that generate